# E-commerce Sales Prediction - Exploratory Data Analysis

This notebook contains comprehensive exploratory data analysis for the e-commerce sales prediction project. We'll analyze various aspects of the data to understand patterns and relationships that can help in predicting weekly sales.

## Setup and Data Loading

In [ ]:
!pip install pyspark
!pip install findspark
!pip install seaborn

: 

In [ ]:
!brew install git-lfs
!git lfs install
!sudo chmod -R 777 /workspace/big-data/.git/lfs/
!git lfs pull


In [ ]:
# Import required libraries
import findspark
findspark.init()

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, when, count, mean, stddev
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Import project utilities
import sys
sys.path.append('../src')

# Set up plotting style
plt.style.use('ggplot')
sns.set_palette('husl')

In [ ]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("SalesPrediction_EDA") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

# Load the dataset
df = spark.read.csv('../data/raw/BaSalam.products.csv', header=True, inferSchema=True)

# Display basic information
print("Dataset Overview:")
print(f"Number of records: {df.count():,}")
print(f"Number of features: {len(df.columns)}")
print("\nSchema:")
df.printSchema()

## Helper Functions

In [ ]:
"""
Utility functions for the sales prediction project.
"""

import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import col, count, mean, stddev
import numpy as np
import pandas as pd


def plot_missing_values(df):
    """
    Plot missing values analysis.
    
    Args:
        df: Spark DataFrame
    """
    total_count = df.count()
    missing_counts = []
    
    for column in df.columns:
        missing_count = df.filter(col(column).isNull()).count()
        missing_percentage = (missing_count / total_count) * 100
        missing_counts.append({
            'column': column,
            'missing_percentage': missing_percentage
        })
    
    missing_df = pd.DataFrame(missing_counts)
    missing_df = missing_df.sort_values('missing_percentage', ascending=True)
    
    plt.figure(figsize=(10, 6))
    plt.barh(missing_df['column'], missing_df['missing_percentage'])
    plt.xlabel('Missing Percentage')
    plt.title('Missing Values Analysis')
    plt.tight_layout()
    plt.show()

def plot_correlation_matrix(df, columns):
    """
    Plot correlation matrix for specified columns.
    
    Args:
        df: Spark DataFrame
        columns: List of column names to include in correlation matrix
    """
    correlation_data = df.select(columns).toPandas()
    correlation_matrix = correlation_data.corr()
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
    plt.title('Correlation Matrix')
    plt.tight_layout()
    plt.show()

def detect_outliers(df, column, threshold=3):
    """
    Detect outliers using z-score method.
    
    Args:
        df: Spark DataFrame
        column: Column name to check for outliers
        threshold: Z-score threshold for outlier detection
        
    Returns:
        DataFrame: DataFrame with outlier indicators
    """
    stats = df.select(
        mean(col(column)).alias('mean'),
        stddev(col(column)).alias('stddev')
    ).collect()[0]
    
    mean_val = stats['mean']
    stddev_val = stats['stddev']
    
    return df.withColumn(
        f'{column}_zscore',
        (col(column) - mean_val) / stddev_val
    ).withColumn(
        f'{column}_is_outlier',
        (col(f'{column}_zscore').abs() > threshold)
    )

def evaluate_regression_model(predictions, label_col="label", prediction_col="prediction"):
    """
    Evaluate regression model performance.
    
    Args:
        predictions: DataFrame with actual and predicted values
        label_col: Name of the label column
        prediction_col: Name of the prediction column
        
    Returns:
        dict: Dictionary of evaluation metrics
    """
    # Calculate MSE
    mse = predictions.select(
        ((col(prediction_col) - col(label_col)) ** 2).alias("squared_error")
    ).agg({"squared_error": "avg"}).collect()[0][0]
    
    # Calculate RMSE
    rmse = np.sqrt(mse)
    
    # Calculate R-squared
    total_variance = predictions.select(
        (col(label_col) - predictions.select(mean(label_col)).collect()[0][0]) ** 2
    ).agg({"label": "sum"}).collect()[0][0]
    
    residual_variance = predictions.select(
        (col(prediction_col) - col(label_col)) ** 2
    ).agg({prediction_col: "sum"}).collect()[0][0]
    
    r2 = 1 - (residual_variance / total_variance)
    
    # Calculate MAE
    mae = predictions.select(
        (abs(col(prediction_col) - col(label_col))).alias("abs_error")
    ).agg({"abs_error": "avg"}).collect()[0][0]
    
    return {
        "mse": mse,
        "rmse": rmse,
        "r2": r2,
        "mae": mae
    }

def plot_actual_vs_predicted(predictions, label_col="label", prediction_col="prediction"):
    """
    Plot actual vs predicted values.
    
    Args:
        predictions: DataFrame with actual and predicted values
        label_col: Name of the label column
        prediction_col: Name of the prediction column
    """
    pred_data = predictions.select(label_col, prediction_col).toPandas()
    
    plt.figure(figsize=(10, 6))
    plt.scatter(pred_data[label_col], pred_data[prediction_col], alpha=0.5)
    plt.plot([pred_data[label_col].min(), pred_data[label_col].max()],
             [pred_data[label_col].min(), pred_data[label_col].max()],
             'r--', lw=2)
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title('Actual vs Predicted Values')
    plt.tight_layout()
    plt.show()

def save_model_metrics(metrics, model_name, output_path):
    """
    Save model evaluation metrics to a file.
    
    Args:
        metrics: Dictionary of evaluation metrics
        model_name: Name of the model
        output_path: Path to save the metrics
    """
    with open(output_path, 'a') as f:
        f.write(f"\n{model_name} Performance Metrics:\n")
        for metric, value in metrics.items():
            f.write(f"{metric}: {value:.4f}\n")
        f.write("-" * 50 + "\n") 

## 1. Missing Data Analysis

In [ ]:
import pandas as pd
# Analyze missing values
plot_missing_values(df)

# Get detailed missing value statistics
missing_stats = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).toPandas()
missing_stats = missing_stats.T.reset_index()
missing_stats.columns = ['Column', 'Missing Count']
missing_stats['Missing Percentage'] = (missing_stats['Missing Count'] / df.count()) * 100
missing_stats = missing_stats.sort_values('Missing Percentage', ascending=False)

print("\nDetailed Missing Value Analysis:")
print(missing_stats[missing_stats['Missing Percentage'] > 0])

## 2. Descriptive Statistics

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import NumericType, StringType, BooleanType, IntegerType

# Assuming `df` is your DataFrame
df_schema = df.dtypes

# Lists to hold the column names
numerical_columns = []
categorical_columns = []
binary_columns = []

# Loop through the schema and classify the columns
for column, dtype in df_schema:
    if isinstance(df.schema[column].dataType, NumericType):
        numerical_columns.append(column)
    elif isinstance(df.schema[column].dataType, StringType):
        categorical_columns.append(column)
    elif isinstance(df.schema[column].dataType, BooleanType):
        binary_columns.append(column)

# Show the results
print("Numerical Columns:", numerical_columns)
print("Categorical Columns:", categorical_columns)
print("Binary Columns:", binary_columns)

### Changing the currency to EGP

In [ ]:
from pyspark.sql.functions import col

# Define the exchange rate
exchange_rate = 0.00038  # 1 Iranian Rial = 0.00038 Egyptian Pound

# Update the 'price', 'primaryPrice', 'vendor_freeShippingToIran', and 'vendor_freeShippingToSameCity' columns
df = df.withColumn('price', col('price') * exchange_rate) \
       .withColumn('primaryPrice', col('primaryPrice') * exchange_rate) \
       .withColumn('vendor_freeShippingToIran', col('vendor_freeShippingToIran') * exchange_rate) \
       .withColumn('vendor_freeShippingToSameCity', col('vendor_freeShippingToSameCity') * exchange_rate)


In [ ]:
# Count the occurrences of each unique value in 'sales_count_week'
sales_count_week_distribution = df.groupBy('sales_count_week').count().orderBy('sales_count_week')

# Show the result
sales_count_week_distribution.show()


In [ ]:
# Count the occurrences of each unique value in 'sales_count_week'
sales_count_week_distribution = df.groupBy('_score').count().orderBy('_score')

# Show the result
sales_count_week_distribution.show()


In [ ]:
# Get the number of unique product names
unique_product_count = df.select('name').distinct().count()

print(f"Number of unique product names: {unique_product_count}")


In [ ]:
# Calculate descriptive statistics for numerical features
numeric_stats = df.select(numerical_columns).describe().toPandas()
print("Numerical Features Statistics:")
display(numeric_stats)

# Display distribution plots for numerical features
# numerical_data = df.select(numerical_columns).toPandas()

# fig, axes = plt.subplots(3, 3, figsize=(15, 15))
# axes = axes.ravel()

# for idx, col in enumerate(numerical_columns):
#     if idx < len(axes):
#         sns.histplot(data=numerical_data, x=col, ax=axes[idx])
#         axes[idx].set_title(f'Distribution of {col}')

# plt.tight_layout()
# plt.show()

In [ ]:
# Get summary statistics for all columns
df_summary = df.describe()

# Filter for categorical (string) columns
categorical_columns = [field.name for field in df.schema.fields if isinstance(field.dataType, StringType)]

# Get summary statistics only for categorical columns
df_categorical_summary = df.select(categorical_columns).describe()

# Show the result
df_categorical_summary.show()


## 3. Sales Analysis

In [ ]:
# Analyze relationship between sales, stock, and price
sales_analysis = df.select('_score', 'stock', 'price', 'primaryPrice').toPandas()

fig, axes = plt.subplots(2, 2, figsize=(15, 15))

# Sales vs Stock
sns.scatterplot(data=sales_analysis, x='stock', y='_score', ax=axes[0,0], alpha=0.5)
axes[0,0].set_title('Score vs Stock')

# Sales vs Price
sns.scatterplot(data=sales_analysis, x='price', y='_score', ax=axes[0,1], alpha=0.5)
axes[0,1].set_title('Score vs Price')

# Sales vs Primary Price
sns.scatterplot(data=sales_analysis, x='primaryPrice', y='_score', ax=axes[1,0], alpha=0.5)
axes[1,0].set_title('Score vs Primary Price')

plt.tight_layout()
plt.show()

## 4. Category Analysis

In [ ]:
# Get the number of unique categories
unique_categories = df.select('categoryTitle').distinct().count()

print(f"Number of unique categories: {unique_categories}")

# Get all unique category titles
unique_category_titles = df.select('categoryTitle').distinct().collect()

# Display the unique categories
for category in unique_category_titles:
    print(category['categoryTitle'])



In [ ]:
# Translation dictionary for category titles
# Updated Translation dictionary for category titles
category_translation = {
    'گام شمار': 'Pedometer',
    'کتاب چاپی': 'Printed Books',
    'طلق موتور': 'Motor Oil',
    'کفش و دمپایی زنانه': 'Women\'s Shoes and Slippers',
    'غذای ماهی و میگو': 'Fish and Shrimp Food',
    'کفش، دمپایی مردانه': 'Men\'s Shoes and Slippers',
    'بذر و تخم گیاهان': 'Seeds and Plant Seeds',
    'ذخیره سازی مبتنی بر نوار': 'Tape-based Storage',
    'عطر و ادکلن زنانه و مردانه': 'Women\'s and Men\'s Perfume and Cologne',
    'سایر': 'Other',
    'ادویه': 'Spices',
    'زیورآلات زنانه': 'Women\'s Jewelry',
    'لباس زیر زنانه': 'Women\'s Lingerie',
    'گیاهان دارویی': 'Medicinal Plants',
    'مانتو و تونیک': 'Manto and Tunic'
}


# Analyze sales by category with product count threshold
category_analysis = df.groupBy('categoryTitle').agg(
    F.avg('_score').alias('avg_score'),
    F.count('*').alias('product_count'),
    F.avg('price').alias('avg_price'),
    F.avg('rating_average').alias('avg_rating')
).toPandas()

# Apply threshold to filter out categories with less than 5000 products
category_analysis = category_analysis[category_analysis['product_count'] >= 5000]

# Translate category titles
category_analysis['categoryTitle'] = category_analysis['categoryTitle'].map(category_translation).fillna(category_analysis['categoryTitle'])

# Plot top categories by average score
plt.figure(figsize=(15, 6))
top_categories = category_analysis.nlargest(10, 'avg_score')
sns.barplot(data=top_categories, x='categoryTitle', y='avg_score')
plt.xticks(rotation=45, ha='right')
plt.title('Average Weekly Sales by Top 10 Categories (with 5000+ Products)')
plt.tight_layout()
plt.show()

# Display category statistics
print("\nCategory Statistics:")
display(category_analysis.sort_values('avg_score', ascending=False).head(10))


## 5. Shipping and Delivery Analysis

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pyspark.sql.functions as F

# Analyze impact of free shipping and delivery options
shipping_features = ['isFreeShipping', 'has_delivery',
                     'vendor_freeShippingToIran', 
                     'vendor_freeShippingToSameCity']

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

for idx, feature in enumerate(shipping_features):
    # Group by feature and calculate the stats
    shipping_stats = df.groupBy(feature) \
        .agg(F.avg('_score').alias('avg_score'),
             F.count('*').alias('count')) \
        .toPandas()
    
    # Handle binning for continuous variables
    if feature in ['vendor_freeShippingToIran', 'vendor_freeShippingToSameCity']:
        # Get min and max values
        min_val = max(0, shipping_stats[feature].min())  # Ensure minimum is 0
        max_val = shipping_stats[feature].max()
        
        # Create 10 bins starting from 0
        bins = np.linspace(min_val, max_val, 11)  # 11 edges to create 10 bins
        
        # Create the binned column
        shipping_stats[feature + '_binned'] = pd.cut(shipping_stats[feature], 
                                                    bins=bins,
                                                    include_lowest=True)
        
        # Update the plot x-axis with binned feature
        x_feature = feature + '_binned'
    else:
        # For other features (like 'isFreeShipping'), just use them directly
        x_feature = feature
    
    # Handle missing values
    shipping_stats = shipping_stats.dropna(subset=[x_feature])
    
    # Plot the barplot
    sns.barplot(data=shipping_stats, x=x_feature, y='avg_score', ax=axes[idx])
    axes[idx].set_title(f'Average Scores by {feature}')
    axes[idx].set_xticklabels(axes[idx].get_xticklabels(), rotation=45)

plt.tight_layout()
plt.show()

## 6. Vendor Analysis

6 Determine which vendors receive the highest customer satisfaction ratings. 
Are there specific vendors that consistently receive positive feedback from 
customers? 
rating_average, 
rating_count, 
vendor_name 

In [ ]:
# Analyze vendor performance
vendor_analysis = df.groupBy('vendor_name') \
    .agg(F.avg('_score').alias('avg_score'),
         F.avg('rating_average').alias('avg_rating'),
         F.count('*').alias('product_count')) \
    .filter('product_count >= 10') \
    .toPandas()

# Plot vendor performance
plt.figure(figsize=(12, 8))
plt.scatter(vendor_analysis['avg_rating'], 
           vendor_analysis['avg_score'],
           s=vendor_analysis['product_count'],
           alpha=0.6)
plt.xlabel('Average Rating')
plt.ylabel('Average Weekly Sales')
plt.title('Vendor Performance: Sales vs Ratings (size = product count)')

# Add annotations for top performers
top_vendors = vendor_analysis.nlargest(5, 'avg_score')
for _, vendor in top_vendors.iterrows():
    plt.annotate(vendor['vendor_name'],
                 (vendor['avg_rating'], vendor['avg_sales']))

plt.tight_layout()
plt.show()

## 7. Correlation Analysis

In [ ]:
# Calculate and plot correlation matrix
plot_correlation_matrix(df, numerical_columns)

# Calculate detailed correlations with sales
correlations = []
for feature in numerical_columns:
    correlation = df.stat.corr('sales_count_week', feature)
    correlations.append({
        'feature': feature,
        'correlation': correlation
    })

correlations_df = pd.DataFrame(correlations)
correlations_df = correlations_df.sort_values('correlation', key=abs, ascending=False)

print("\nCorrelations with sales_count_week:")
display(correlations_df)

## 8. Outlier Detection

In [ ]:
# Create box plots for numerical features
plt.figure(figsize=(15, 6))
numerical_data.boxplot(column=numerical_columns, figsize=(15, 6))
plt.xticks(rotation=45)
plt.title('Box Plots of Numerical Features')
plt.tight_layout()
plt.show()

# Detect outliers using Z-score method
outlier_stats = {}
for feature in numerical_columns:
    df_with_outliers = detect_outliers(df, feature)
    outlier_count = df_with_outliers.filter(col(f'{feature}_is_outlier')).count()
    outlier_percentage = (outlier_count / df.count()) * 100
    outlier_stats[feature] = {
        'outlier_count': outlier_count,
        'outlier_percentage': outlier_percentage
    }

print("\nOutlier Statistics (|z-score| > 3):")
outlier_df = pd.DataFrame.from_dict(outlier_stats, orient='index')
display(outlier_df)

## 9. Key Insights Summary

Based on the analysis above, here are the key insights:

1. **Missing Data**:
   - [Will be filled based on actual analysis]

2. **Sales Patterns**:
   - Relationship between sales and stock levels
   - Impact of pricing on sales
   - Effect of discounts

3. **Category Performance**:
   - Top performing categories
   - Category-wise pricing strategies

4. **Shipping Impact**:
   - Effect of free shipping on sales
   - Delivery options influence

5. **Vendor Analysis**:
   - Top performing vendors
   - Relationship between ratings and sales

6. **Correlations**:
   - Strong predictors of sales
   - Feature relationships

7. **Outliers**:
   - Distribution of extreme values
   - Impact on modeling strategy